# Data Preprocessing 

In [1]:
#Author: Sandesh Padiyar
#importing required modules
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import spacy

In [2]:
chunk=pd.read_csv('food_recommendation.csv',chunksize=1000)

In [3]:
df=pd.concat(chunk)

In [4]:
df.head()

,ingredients,url,partition,title,id,instructions,type
0,5 cups water 4 cups frozen hash brown potatoes...,http://www.epicurious.com/recipes/food/views/m...,train,mashed potato bake,165413dfa5,preheat the oven to 350 degrees. bring the wat...,0
1,2 whole sheets of lavash bread 1 teaspoon grou...,http://tastykitchen.com/recipes/appetizers-and...,train,cinnamon and sugar lavash chips,1654357cd3,preheat oven to 350 degrees f. line a baking s...,0
2,1 c. sugar 1/2 c. butter 1/2 c. white corn syr...,http://cookeatshare.com/recipes/almond-toffee-...,train,almond toffee popcorn recipe,165438a19c,"in heavy saucepan, combine sugar, butter, corn...",0
3,2 tablespoons ground paprika 1 tablespoon grou...,http://allrecipes.com/recipe/blackened-salmon-...,val,blackened salmon fillets,1654587b71,"in a small bowl, mix paprika, cayenne pepper, ...",1
4,2 lb ground turkey 1 quart speghetti sauce 1 b...,https://cookpad.com/us/recipes/349446-baked-ziti,train,baked ziti,16545e6d7e,make your pasta and sauce like you normally wo...,1


In [5]:
#Finding same name dishes (Note: the ingrediants are different , therefore same dishes with different style of preparation)
dish_dup=df[df.duplicated(subset='title')]
print(dish_dup.title)
df[df.title=='blueberry crisp']

494         blueberry crisp
540              baked ziti
987        hot buttered rum
1040            tomato soup
1986    asparagus casserole
               ...         
9829       creamy fruit dip
9880         raspberry bars
9891       watermelon salsa
9960    italian pasta salad
9963      hot artichoke dip
Name: title, Length: 139, dtype: object


,ingredients,url,partition,title,id,instructions,type
79,2 sticks margarine 2 cups all-purpose flour 1 ...,http://www.foodgeeks.com/recipes/16110,train,blueberry crisp,16591c7b5c,melt butter and mix with flour and pecans. pac...,0
494,12 cup sugar 13 cup flour 1 teaspoon cinnamon ...,http://www.food.com/recipe/blueberry-crisp-83767,train,blueberry crisp,167228d018,"in a small bowl, combine sugar, flour, cinnamo...",0


<strong>Observation: </strong>Finding dishes with same name (Note: the ingrediants are different , therefore same dishes with different style of preparation)

In [6]:
sum(df.id.duplicated())

0

In [7]:
#dropping url as it does not provide any useful information (type column will be modified later)
new_df=df.drop(['url','type'],axis=1)
new_df.head()

,ingredients,partition,title,id,instructions
0,5 cups water 4 cups frozen hash brown potatoes...,train,mashed potato bake,165413dfa5,preheat the oven to 350 degrees. bring the wat...
1,2 whole sheets of lavash bread 1 teaspoon grou...,train,cinnamon and sugar lavash chips,1654357cd3,preheat oven to 350 degrees f. line a baking s...
2,1 c. sugar 1/2 c. butter 1/2 c. white corn syr...,train,almond toffee popcorn recipe,165438a19c,"in heavy saucepan, combine sugar, butter, corn..."
3,2 tablespoons ground paprika 1 tablespoon grou...,val,blackened salmon fillets,1654587b71,"in a small bowl, mix paprika, cayenne pepper, ..."
4,2 lb ground turkey 1 quart speghetti sauce 1 b...,train,baked ziti,16545e6d7e,make your pasta and sauce like you normally wo...


# Text Preprocessing

In [8]:
new_df.shape

(10002, 5)

In [9]:
#printing random rows to get idea of preprocessing to be done
for i in np.random.randint(0,10000,10):
    print(i,'...',df.loc[i,'ingredients'])


3911 ... 2 cups flour 2 tablespoons flour 1 teaspoon baking powder 14 teaspoon salt 18 teaspoon baking soda 1 cup sugar 2 large eggs 2 large egg whites 1 tablespoon almond extract 1 teaspoon almond extract 2 -3 ounces sliced almonds, lightly toasted and coarsly chopped 2 -4 tablespoons mini chocolate chips
7805 ... 1-1/2 cups (350 ml) flour 1/2 tsp (2 ml) salt 1-1/2 tbsp (20 ml) sugar 1/2 cup (125 ml) vegetable oil 2 tbsp (30 ml) milk
5902 ... 12 cup raisins 2 tablespoons orange juice or 2 tablespoons apple juice 8 ounces cream cheese or 8 ounces soft fresh goat cheese 14 cup chopped fresh basil 14 cup sun-dried tomato packed in oil, drained and chopped fine 1 garlic clove, peeled and minced 14 teaspoon crushed red pepper flakes 3 heads belgian endive or 3 heads crackers 4 basil sprigs (to garnish)
9039 ... 1 from 3 fruit daidai or yuzu juice 1 the same amount as the citrus juice vinegar 1 the same amount as the citrus juice and vinegar combined soy sauce 1 dash umami seasoning 1 tsp s

<strong>Observation: </strong>The punctuations , numbers can be removed as those factors do not play role in a user's reason to like the ingredient

In [10]:
#classifying to veg and non-veg based on most common words (more words to be added ..not 100% accurate) based on ingredients
non_veg = ['turkey', 'chicken', 'egg', 'fish', 'milk', 'beef', 'pork', 'salmon','prawn','squid','crab','lobster']

new_df['is_veg']=np.nan
new_df['lactose_content']=np.nan
lst = []               
for i in new_df.ingredients:
    lst.append(i)         #putting all title values into the list 'lst'
    
    

#for nonveg ---> False   for veg ---> True

for i in range(len(lst)):
    count = 0
    j = lst[i].split(" ")   #getting words for each list values
    for k in j:
        if  k in non_veg:
            new_df['is_veg'][i] = False
            break
        else:
            count = count+1      #traversing through j into the last word, if last word is also not in 'non_veg list' then 0
            if(count == len(j)):
                new_df['is_veg'][i] = True
                break
            continue

<ipython-input-10-575fd1652a34>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['is_veg'][i] = True
E:\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-10-575fd1652a34>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['is_veg'][i] = False


In [12]:
new_df.head()

,ingredients,partition,title,id,instructions,is_veg,lactose_content
0,5 cups water 4 cups frozen hash brown potatoes...,train,mashed potato bake,165413dfa5,preheat the oven to 350 degrees. bring the wat...,True,NaN
1,2 whole sheets of lavash bread 1 teaspoon grou...,train,cinnamon and sugar lavash chips,1654357cd3,preheat oven to 350 degrees f. line a baking s...,True,NaN
2,1 c. sugar 1/2 c. butter 1/2 c. white corn syr...,train,almond toffee popcorn recipe,165438a19c,"in heavy saucepan, combine sugar, butter, corn...",True,NaN
3,2 tablespoons ground paprika 1 tablespoon grou...,val,blackened salmon fillets,1654587b71,"in a small bowl, mix paprika, cayenne pepper, ...",False,NaN
4,2 lb ground turkey 1 quart speghetti sauce 1 b...,train,baked ziti,16545e6d7e,make your pasta and sauce like you normally wo...,False,NaN


In [13]:
#Adding columns for lactose content for lactose intolerant people based on ingredients(I considered this as important
#according to stats almost 65% of population of world have lactose intolerance issues)
lactose=['milk','cheese','butter','yogurt','butter','cream']
new_df['lactose_content']=np.nan
lst = []               
for i in new_df.ingredients:
    lst.append(i)         #putting all title values into the list 'lst'
    
    

#for lactose ---> True   for no_lactose ---> False

for i in range(len(lst)):
    count = 0
    j = lst[i].split(" ")   #getting words for each list values
    for k in j:
        if  k in lactose:
            new_df['lactose_content'][i] = True
            break
        else:
            count = count+1      #traversing through j into the last word, if last word is also not in 'non_veg list' then 0
            if(count == len(j)):
                new_df['lactose_content'][i] = False
                break
            continue

<ipython-input-13-40bbea068892>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['lactose_content'][i] = True
E:\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [14]:
new_df.head()

,ingredients,partition,title,id,instructions,is_veg,lactose_content
0,5 cups water 4 cups frozen hash brown potatoes...,train,mashed potato bake,165413dfa5,preheat the oven to 350 degrees. bring the wat...,True,True
1,2 whole sheets of lavash bread 1 teaspoon grou...,train,cinnamon and sugar lavash chips,1654357cd3,preheat oven to 350 degrees f. line a baking s...,True,False
2,1 c. sugar 1/2 c. butter 1/2 c. white corn syr...,train,almond toffee popcorn recipe,165438a19c,"in heavy saucepan, combine sugar, butter, corn...",True,True
3,2 tablespoons ground paprika 1 tablespoon grou...,val,blackened salmon fillets,1654587b71,"in a small bowl, mix paprika, cayenne pepper, ...",False,False
4,2 lb ground turkey 1 quart speghetti sauce 1 b...,train,baked ziti,16545e6d7e,make your pasta and sauce like you normally wo...,False,False


<h3>Word substitution (ex: 're = are ,etc)

In [15]:
#Reason: Seeing random reviews it was clear that there was mix of shortforms and the expansions, though the meaning of both is same)

def pre_processing(phrase):
    phrase = re.sub(r"tbsp.", "tablespoon ", phrase)
    phrase = re.sub(r"tablespoons", "tablespoon ", phrase)
    phrase = re.sub(r"tsp.", "teaspoon ", phrase)
    phrase = re.sub(r"oz.", "ounce ", phrase)
    phrase = re.sub(r"teaspoons", "tablespoon ", phrase)
    phrase = re.sub(r"\b\d+\b", '', phrase) #remving digits
    phrase = phrase.translate(str.maketrans('', '', string.punctuation)) #removing punctuations
    phrase=re.sub(' +', ' ', phrase) #removing unwanted spaces
    return phrase

In [16]:
new_df.ingredients=new_df['ingredients'].apply(pre_processing)
new_df.instructions=new_df['instructions'].apply(pre_processing)

# Stop word Removal

In [17]:
#defining the set of stopwords aside from the general stop words in english (source=https://gist.github.com/sebleier/554280)
word_set=set(['cup','cups','teaspoon','tablespoon','teaspoons','tablespoons','spoon','ounce','c','store','tbsp','tsp','lbs','inches',
              'into','divided','tbs','pounds','possibly','with','lb','cans','optional','cut','chop','ounces','chopped','optional','pound',
             'can','or','if','but','or',"i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", 
              "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself",
              "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those",
              "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", 
              "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for",
              "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", 
              "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", 
              "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", 
              "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", 
              "don", "should", "now",'a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u'
              ,'v','w','x','y','z','also'])

In [18]:
for i in range(len(new_df.ingredients)):
    words = [word for word in new_df.ingredients[i].strip().split(' ') if word.lower() not in word_set]
    new_df.ingredients[i] = " ".join(words)
    
new_df.ingredients=new_df['ingredients'].apply(pre_processing)

In [19]:
new_df.ingredients

0        water frounce hash brown potatoes one package ...
1          whole sheets lavash bread ground cinnamon sugar
2        sugar butter white corn syrup water toasted al...
3        ground paprika ground cayenne pepper onion pow...
4        ground turkey quart speghetti sauce box ziti p...
                               ...                        
9997     butter shortening sugar vanilla eggs cocoa flo...
9998     crispy chicken homade brought lettuce tomatoes...
9999     hanger steak inch cubes trimmed needed irish s...
10000    unsalted butter roasted unsalted peanuts sugar...
10001    hazelnuts melted butter good quality semisweet...
Name: ingredients, Length: 10002, dtype: object

In [20]:
for i in range(len(new_df.instructions)):
    words = [word for word in new_df.instructions[i].strip().split(' ') if word.lower() not in word_set]
    new_df.instructions[i] = " ".join(words)
    
new_df.instructions=new_df['instructions'].apply(pre_processing)

In [21]:
new_df.instructions

0        preheat oven degrees bring water boil add hash...
1        preheat oven degrees line baking sheet aluminu...
2        heavy saucepan combine sugar butter corn syrup...
3        small bowl mix paprika cayenne pepper onion po...
4        make pasta sauce like normally would adding ve...
                               ...                        
9997     generously grease flour inch round 13x9 cake p...
9998     ingriedients personal size add ingredients pla...
9999     pair hearty stew dry irish stout marinade gall...
10000    position oven rack center oven preheat oven 35...
10001    butter springform pan process nuts food proces...
Name: instructions, Length: 10002, dtype: object

In [22]:
#displaying random ingredient 
for i in np.random.randint(0,10000,10):
    print(new_df.loc[i,'ingredients'])

stewing lamb quarts water leek salt sprig fresh parsley pearl barley carrot peppercorns bay leaves fresh parsley
extra virgin olive oil lemon juice sea salt cracked pepper spinach butter lettuce crumbled feta
chicken skinned equivalent chicken pieces onion piece gingerroot peeled sliced garlic cloves peppercorn salt thai fish sauce soy sauce dried rice noodles baby spinach cilantro limes
box yellow cake mix quickcooking oats oldfashioned nuts walnuts butter margarine softened egg sweetened condensed milk fresh grated orange zest shredded coconut semisweet chocolate chips mms
bread flour fast rise yeast sugar kosher salt less use regular salt olive oil hot water degrees
water ice cubes instant coffee dark cocoa powdered sugar xanthan gum
whole boneless chicken breasts pounded flattened inch thickness flour salt white pepper ground nutmeg italian seasoning large egg water dry breadcrumbs freshly grated parmesan cheese butter olive oil dry white wine lemon wedge
lean ground beef macaroni 

In [23]:
for i in np.random.randint(0,10000,5): #random 5 instructions
    print(new_df.loc[i,'instructions'])

put flour salt bowl make well pour slightly beaten eggs well stir fork mixed stir rum add milk slowly stirring constantly fork batter smooth lumps need broken blended let batter sit minutes proceeding brush crepe heavy frying pan size melted butter pan hot pour enough batter thinly cover bottom pan tilt pan base completely covered batter cook crepe golden bottom minute flip metal spatula cook second side golden transfer crepe plate cover foil keep warm repeat cooking process batter used unfilled crepes well wrapped frounce recipe halved use two eggs
saute onion carrots celery potatoes thyme large pot onions transparent add garlic mushrooms saute minutes add broths barley cook covered minutes barley cooked veggies soft add salt pepper taste stew adjust seasonings like serve hot crusty bread crackers whatever like stew bon appetit
peel grate carrots make vinaigrette stirring together small bowl red wine vinegar fresh lemon juice salt freshground black pepper whisk olive oil taste adjust 

<h3>Lammetizing

In [24]:
import nltk
from nltk.stem import WordNetLemmatizer

In [25]:
nltk.download('wordnet')
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sandesh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
for i in range(len(new_df.ingredients)):
    words_ingr = [lemmatizer.lemmatize(word) for word in new_df.ingredients[i].strip().split(' ')]
    words_instr= [lemmatizer.lemmatize(word) for word in new_df.instructions[i].strip().split(' ')]
    new_df.ingredients[i] = " ".join(words_ingr)
    new_df.instructions[i]=" ".join(words_instr)


<h3> Old Dataset before Preprocessing

In [38]:
for i in range(5): #random 5 instructions
    print(df.loc[i,'instructions'])

preheat the oven to 350 degrees. bring the water to a boil, add the hash browns, and cook for 5 minutes; drain. prepare the mashed-potato mix according to package directions. soften the butter and cream cheese together in the microwave. in a large bowl, mix together the hash browns, mashed potatoes, butter and cream cheese, jack cheese, sour cream, garlic salt, salt, and pepper. place in a greased 2-quart casserole. bake for 25 to 35 minutes, or until golden on top. sprinkle the onion rings over the casserole and bake for about 5 to 8 minutes longer, or until onions are golden brown.
preheat oven to 350 degrees f. line a baking sheet with aluminum foil and spray with cooking spray. cut one sheet of lavash bread into 30 squares. do the same with the other pieces. i found cutting them with a pizza cutter works quick and easy. line the lavash bread on baking sheet. try not to overlap them. lightly spray with cooking spray. in a small bowl add cinnamon and sugar. with a sifter, add the sug

<h3> After Preprocessing

In [37]:
for i in range(5): #random 5 instructions
    print(new_df.loc[i,'instructions'])

preheat oven degree bring water boil add hash brown cook minute drain prepare mashedpotato mix according package direction soften butter cream cheese together microwave large bowl mix together hash brown mashed potato butter cream cheese jack cheese sour cream garlic salt salt pepper place greased quart casserole bake minute golden top sprinkle onion ring casserole bake minute longer onion golden brown
preheat oven degree line baking sheet aluminum foil spray cooking spray one sheet lavash bread square piece found cutting pizza cutter work quick easy line lavash bread baking sheet try overlap lightly spray cooking spray small bowl add cinnamon sugar sifter add sugar cinnamon shake lavash bread place oven bake minute flip cracker bake minute
heavy saucepan combine sugar butter corn syrup water almond cook moderate heat candy thermometer add vanilla stir well pour popped corn
small bowl mix paprika cayenne pepper onion powder salt white pepper black pepper thyme basil oregano brush salmo